## **First of all**
create folder "cyborg" on google drive
Save file spectation_data.npz you got from [expert_recorder.py](https://github.com/uwanderer91/Cyborg-T-800-revival/blob/main/expert_recorder.py) in this folder

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distributions
from torch.utils.data import Dataset, DataLoader
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## **Creating the model.**
In our case we have 32 filters on the first layer and 64 filters on the second layer. In result we have 64 high-level 4x4 features checker

In [2]:
class NN(nn.Module):
    def __init__(self, input_channels, num_actions):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 32, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, num_actions)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def save(self, filepath='/content/drive/My Drive/cyborg/model.npz'):
        torch.save(self.state_dict(), filepath)
        print(f"Model saved: {filepath}")

    def load(self, filepath='/content/drive/My Drive/cyborg/model.npz'):
        self.load_state_dict(torch.load(filepath))
        print(f"Model loaded: {filepath}")

## **Load and save trajectory numpy data(without rewards and dones)**


1.   states
2.   actions



In [3]:
import numpy as np
import torch

def save_spectation(spec_data, filename='/content/drive/My Drive/cyborg/spectation_data.npz'):
    np.savez(filename, **spec_data)
    print("Saved in "+filename)

def load_spectation(filename='/content/drive/My Drive/cyborg/spectation_data.npz'):
    spec_data = np.load(filename)
    expert_data = {}

    for key in spec_data.files:
        expert_data[key] = spec_data[key]

    spec_data.close()
    return expert_data

## **Train the model**

1.   Load our saved expert data, which we got from [expert_recorder.py](https://github.com/uwanderer91/Cyborg-T-800-revival/blob/main/expert_recorder.py)
2.   Shape into correct dimension(in our case (tensor, channel, height, width))
3.   train our model with Cross Entropy

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distributions
import numpy as np

def train_imitation_learning(model, expert_data, epochs=50, lr=1e-3):
    observations = torch.FloatTensor(expert_data['observations'])
    actions = torch.LongTensor(expert_data['actions'])

    dataset = torch.utils.data.TensorDataset(observations, actions)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for batch_obs, batch_actions in dataloader:
            logits = model(batch_obs)
            loss = criterion(logits, batch_actions)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = logits.argmax(dim=1)
            correct += (preds == batch_actions).sum().item()
            total += batch_actions.size(0)

        accuracy = 100 * correct / total
        avg_loss = total_loss / len(dataloader)

        if epoch % 10 == 0:
            print(f'Epoch {epoch:2d} | Loss: {avg_loss:.4f} | Acc: {accuracy:.2f}%')

    model.save()

    return model

if __name__ == "__main__":
    model = NN(
        input_channels=1,
        num_actions=7
    )

    print(f"model parameters: {sum(p.numel() for p in model.parameters()):,}")

    expert_data = load_spectation()
    obs_encoded = np.reshape(expert_data["observations"], (len(expert_data["observations"]), 1, 64, 64))
    expert_data = {
        'observations': obs_encoded,
        'actions': expert_data['actions']
    }

    trained_model = train_imitation_learning(model, expert_data, epochs=100)

model parameters: 188,359
Epoch  0 | Loss: 10.6013 | Acc: 33.42%
Epoch 10 | Loss: 1.2133 | Acc: 53.18%
Epoch 20 | Loss: 1.0262 | Acc: 58.86%
Epoch 30 | Loss: 0.9391 | Acc: 62.79%


KeyboardInterrupt: 

If the code above throws an error, try change execution environment(in my case 2025.07)

At the end we have model.npz in the "cyborg" folder, which can be tested by running [evaluate_model.py](https://github.com/uwanderer91/Cyborg-T-800-revival/blob/main/evaluate_model.py)